# Tolerancing a Beamline
This example shows how to do tolerancing on a optical beamline.

Any parameter of any optical element in the active chain can be toleranced by defining its bounds and running a MonteCarlo analysis

## Usual imports

In [1]:
import ctypes
import sys, os
sys.path.append("../..")
import numpy as np
from scipy.constants import h, c, eV, nano, milli, degree, pi
import pyoptix
hc = h*c/eV
pyoptix.output_notebook()

intialzing SR library
OptiX library initialized


Loading BokehJS ...

## Import of the beamline defined in KB_HELIMAG.ipynb

In order to keep the notebook tidy and quick to run, the definition of the beamline is left to another notebook and the beamline itself is imported via a pickle serialized file.

*Note*: the load_beamline function takes globals() as a second parameter. This is python trickery, user should not pay attention to this argument as use it as is.

*Note2*: The optical element from the definition notebook are instanciated in the memory in variable matching their name in the definition notebook. 
For example, the plane mirror definied as `plan = PlaneMirror(name="M1")` will now the instanciated in an object called M1. A summary of these names is printed during loading.

In [2]:
from pyoptix.classes import load_beamline
load_beamline("KB_HELIMAG_definition.pickle", globals())
p = 1.8 #m
q = 0.06 #m
theta = 20*degree
dist = 0.05 #m

Retrieving beamline KB
Retrieving and setting element gaussSource
Retrieving and setting element M1
Retrieving and setting element VFM
Retrieving and setting element HFM
Retrieving and setting element foc
Chaîne whole KB:
	gaussSource -> M1 -> VFM -> HFM -> foc 


In [3]:
foc.recording_mode = pyoptix.classes.RecordingMode.recording_input

## Adding and editing tolerancing parameters
These parameters are stored as parameter bounds

In [4]:
# for safekeeping :
deltatheta = 0.1e-3 # 100µrad
ds = 0.1e-3 # 100µm
dposKB = 1e-3 # 1mm
source_field = 0.1e-3 # dépointé 100µm

M1.d_theta = {"bounds":[-deltatheta, deltatheta]}
HFM.d_theta = {"bounds":[-deltatheta, deltatheta]}
VFM.d_theta = {"bounds":[-deltatheta, deltatheta]}
HFM.distance_from_previous = {"bounds":[dist-ds, dist+ds]}
VFM.distance_from_previous = {"bounds":[dist-ds, dist+ds]}
M1.distance_from_previous = {"bounds":[1.7-dposKB, 1.7+dposKB]}
gaussSource.d_x =  {"bounds":[-source_field, source_field]}
gaussSource.d_z =  {"bounds":[-source_field, source_field]}

In [5]:
from pyoptix.tolerancing import sensitivity_analysis, display_tolerance_data, apply_tolerance_data
tolerance_data = display_tolerance_data(KB)

properties of gaussSource


Sheet(cells=(Cell(column_end=7, column_start=0, row_end=0, row_start=0, squeeze_column=False, style={'textAlig…

properties of M1


Sheet(cells=(Cell(column_end=7, column_start=0, row_end=0, row_start=0, squeeze_column=False, style={'textAlig…

properties of VFM


Sheet(cells=(Cell(column_end=7, column_start=0, row_end=0, row_start=0, squeeze_column=False, style={'textAlig…

properties of HFM


Sheet(cells=(Cell(column_end=7, column_start=0, row_end=0, row_start=0, squeeze_column=False, style={'textAlig…

properties of foc


Sheet(cells=(Cell(column_end=7, column_start=0, row_end=0, row_start=0, squeeze_column=False, style={'textAlig…

In order to be taken into account, changes made in the tables must be followed by :

In [6]:
apply_tolerance_data(KB, tolerance_data)

## Saving these paremeters in memory for later use 

In [7]:
tolerancing_configuration = KB.save_configuration()

In [8]:
gaussSource.display_properties()

Sheet(cells=(Cell(column_end=7, column_start=0, row_end=0, row_start=0, squeeze_column=False, style={'textAlig…

## Running the Monte Carlo tolerancing

In [9]:
from pyoptix.optimize import find_focus
def mae(beamline):
    diag = foc.get_diagram()
    return diag["X"].std(), diag["Y"].std()
compensator = lambda: find_focus(KB, foc, 20e-9, dimension="xy", adjust_distance=True, verbose=0, tol=1e-6)
res_comp, res = sensitivity_analysis(KB, quality=mae, N_stat=100, N_rays=1000, distribution="uniform", compensator=compensator)

IntProgress(value=0)

In [10]:
print("# uncompensated")
print(f"X RMS size in [{res.minmax[0][0]:.2e}; {res.minmax[1][0]:.2e}] <X> = {res.mean[0]:.2e} sigma = {res.variance[0]:.2e}")
print(f"Y RMS size in [{res.minmax[0][1]:.2e}; {res.minmax[1][1]:.2e}] <Y> = {res.mean[1]:.2e} sigma = {res.variance[1]:.2e}")
print("# compensated")
print(f"X RMS size in [{res_comp.minmax[0][0]:.2e}; {res_comp.minmax[1][0]:.2e}] <X> = {res_comp.mean[0]:.2e} sigma = {res_comp.variance[0]:.2e}")
print(f"Y RMS size in [{res_comp.minmax[0][1]:.2e}; {res_comp.minmax[1][1]:.2e}] <Y> = {res_comp.mean[1]:.2e} sigma = {res_comp.variance[1]:.2e}")

# uncompensated
X RMS size in [8.81e-07; 1.50e-04] <X> = 3.80e-06 sigma = 2.21e-10
Y RMS size in [1.54e-06; 7.78e-05] <Y> = 2.99e-06 sigma = 5.75e-11
# compensated
X RMS size in [8.55e-07; 6.83e-06] <X> = 2.27e-06 sigma = 2.18e-12
Y RMS size in [1.56e-06; 3.32e-06] <Y> = 2.05e-06 sigma = 2.02e-13


## Checking the values taken by the parameter in the latest version of the beamline and saving the beamline

In [11]:
gaussSource.display_properties()

Sheet(cells=(Cell(column_end=7, column_start=0, row_end=0, row_start=0, squeeze_column=False, style={'textAlig…

In [12]:
KB.load_configuration(tolerancing_configuration)
pyoptix.save_beamline(KB, "whole KB", "KB_HELIMAG_tolerancing.pickle")